In [70]:
import gzip
import requests
import pandas as pd
import os
import pickle
import errno
import datetime
from collections import namedtuple

In [50]:
BITMEX_QUOTE_URL = 'https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/quote/'
FILENAME_EXT = '.csv.gz'
# ref: https://www.bitmex.com/app/futuresGuideExamples
MONTH_CODES = {'F': 1,
               'G': 2,
               'H': 3,
               'J': 4,
               'K': 5,
               'M': 6,
               'N': 7,
               'Q': 8,
               'U': 9,
               'V': 10,
               'X': 11,
               'Z': 12}
date = '20180101'
QUOTE_PKL_DIR = os.path.join(os.getcwd(), 'pkl', 'quote')

In [51]:
def get_quote_df(date):
    quote_pkl_file = os.path.join(QUOTE_PKL_DIR, date + '.pkl')
    # try loading df from pickle if exists
    if os.path.exists(quote_pkl_file):
        with open(quote_pkl_file, 'rb') as f:
            quote_df = pickle.load(f)
    # else get CSV from BitMEX and load as df
    else:
        try:
            os.makedirs(quote_pkl_dir)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        resp = requests.get(BITMEX_QUOTE_URL + date + FILENAME_EXT, stream=True)
        if resp.status_code == 200:
            with gzip.open(resp.raw) as raw_resp:
                quote_df = pd.read_csv(raw_resp)
                quote_pkl_file = os.path.join(quote_pkl_dir, date + '.pkl')
                with open(quote_pkl_file, 'wb') as f:
                    pickle.dump(quote_df, f, pickle.HIGHEST_PROTOCOL)
    return quote_df

In [62]:
quote_df = get_quote_df('20180101')
quote_df.head(10)

,timestamp,symbol,bidSize,bidPrice,askPrice,askSize
0,2018-01-01D00:00:02.008398000,XBTUSD,14559,13872.500000,13873.000000,21501
1,2018-01-01D00:00:02.020619000,XBTUSD,14549,13872.500000,13873.000000,21501
2,2018-01-01D00:00:02.024411000,XBTUSD,14539,13872.500000,13873.000000,21501
3,2018-01-01D00:00:02.029607000,XBTUSD,14529,13872.500000,13873.000000,21501
4,2018-01-01D00:00:02.032495000,XBTUSD,14519,13872.500000,13873.000000,21501
5,2018-01-01D00:00:02.044164000,XBTH18,23871,14673.000000,14683.000000,30000
6,2018-01-01D00:00:02.055657000,XBTUSD,14519,13872.500000,13873.000000,24501
7,2018-01-01D00:00:02.069278000,ETHH18,8,0.056880,0.056920,15
8,2018-01-01D00:00:02.069278000,ZECH18,44,0.038318,0.041393,1
9,2018-01-01D00:00:02.254353000,XBTH18,23871,14673.000000,14673.500000,30000


In [73]:
# leave Bitcoin contracts data only
xbt_quote_df = quote_df[quote_df['symbol'].str.startswith('XBT')]
OHLC = namedtuple('OHLC', ('open', 'high', 'low', 'close'))

In [89]:
def get_day_ohlc(df):
    bid_ask_avg_df = df[['askPrice', 'bidPrice']].mean(axis=1)
    o = bid_ask_avg_df.iloc[0]
    h = max(bid_ask_avg_df)
    l = min(bid_ask_avg_df)
    c = bid_ask_avg_df[-1]
    ohlc = OHLC(o, h, l, c)
    return ohlc

In [90]:
day_ohlc = get_day_ohlc(xbt_quote_df[xbt_quote_df['symbol'] == 'XBTUSD'])

In [91]:
day_ohlc

OHLC(open=13872.75, high=14006.0, low=12772.25, close=13476.75)

In [106]:
pd.date_range('2018-01-01', '2018-08-31').to_datetime().strftime('%Y%m%d')

array(['20180101', '20180102', '20180103', '20180104', '20180105',
       '20180106', '20180107', '20180108', '20180109', '20180110',
       '20180111', '20180112', '20180113', '20180114', '20180115',
       '20180116', '20180117', '20180118', '20180119', '20180120',
       '20180121', '20180122', '20180123', '20180124', '20180125',
       '20180126', '20180127', '20180128', '20180129', '20180130',
       '20180131', '20180201', '20180202', '20180203', '20180204',
       '20180205', '20180206', '20180207', '20180208', '20180209',
       '20180210', '20180211', '20180212', '20180213', '20180214',
       '20180215', '20180216', '20180217', '20180218', '20180219',
       '20180220', '20180221', '20180222', '20180223', '20180224',
       '20180225', '20180226', '20180227', '20180228', '20180301',
       '20180302', '20180303', '20180304', '20180305', '20180306',
       '20180307', '20180308', '20180309', '20180310', '20180311',
       '20180312', '20180313', '20180314', '20180315', '201803

In [107]:
def get_quote_dfs(start, end):
    date_range = pd.date_range(start, end).to_datetime().strftime('%Y%m%d')
    quote_dfs = []
    for date in date_range:
        df = get_quote_df(date)
        quote_dfs.append(df)
    return quote_dfs

In [ ]:
quote_dfs = get_quote_dfs('2018-01-01', '2018-03-31')

In [109]:
xbt_quote_df[xbt_quote_df['symbol'] == 'XBTH18']

,timestamp,symbol,bidSize,bidPrice,askPrice,askSize
5,2018-01-01D00:00:02.044164000,XBTH18,23871,14673.0,14683.0,30000
9,2018-01-01D00:00:02.254353000,XBTH18,23871,14673.0,14673.5,30000
12,2018-01-01D00:00:02.389472000,XBTH18,23871,14673.0,14673.5,36000
17,2018-01-01D00:00:03.291989000,XBTH18,23871,14673.0,14673.5,30000
18,2018-01-01D00:00:03.299370000,XBTH18,23871,14673.0,14673.5,20000
27,2018-01-01D00:00:05.461763000,XBTH18,23871,14673.0,14682.5,30000
28,2018-01-01D00:00:05.605438000,XBTH18,23871,14673.0,14673.5,20000
29,2018-01-01D00:00:05.623529000,XBTH18,23871,14673.0,14683.0,20000
30,2018-01-01D00:00:05.650214000,XBTH18,23871,14673.0,14673.5,20000
31,2018-01-01D00:00:05.670224000,XBTH18,23871,14673.0,14683.0,20000
